In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

# Define required input dimensions
input_channels = 3  # Assuming RGB images
input_width = 80  # Update with the actual width
input_height = 80  # Update with the actual height


In [3]:
class SatelliteClassifier(nn.Module):
    def __init__(self, channels, batchNorm=False):
        super(SatelliteClassifier, self).__init__()
        self.conv1 = nn.Conv2d(channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        if batchNorm:
            self.conv2_bn = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        if batchNorm:
            self.conv4_bn = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 20 * 20, 128)
        self.fc2 = nn.Linear(128, 4) # 4 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        if hasattr(self, 'conv2_bn'):
            x = self.conv2_bn(x)
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        if hasattr(self, 'conv4_bn'):
            x = self.conv4_bn(x)
        x = self.pool2(x)
        x = x.view(-1, 64 * (80 // 4) * (80 // 4))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Instantiate the model
model = SatelliteClassifier(input_channels)

# Choose a loss function and optimizer
criterion = nn.CrossEntropyLoss()

optimizers = [
    {'optimizer':optim.SGD(model.parameters(), lr=1, momentum=0.9),"batchNorm":False},
    {'optimizer':optim.SGD(model.parameters(), lr=0.1, momentum=0.9),"batchNorm":True},
    {'optimizer':optim.SGD(model.parameters(), lr=0.01, momentum=0.9),"batchNorm":False},
    {'optimizer':optim.Adam(model.parameters(), lr=1),"batchNorm":False},
    {'optimizer':optim.Adam(model.parameters(), lr=0.1),"batchNorm":True},
    {'optimizer':optim.Adam(model.parameters(), lr=0.01),"batchNorm":False},
]

transform = transforms.Compose([
    transforms.Resize((input_width, input_height)),
    transforms.ToTensor()
])



In [5]:
# Adjust 'root' with the actual path to your dataset
dataset = datasets.ImageFolder(root='./Dataset/data/', transform=transform)



In [8]:

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



1127

In [6]:
for optimizer in optimizers:
    model = SatelliteClassifier(input_channels,optimizer['batchNorm'])
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer['optimizer'].zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer['optimizer'].step()
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += int((predicted == labels).sum())
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, loss.item(), correct / total))

    
    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(optimizer['optimizer'],optimizer['batchNorm'],accuracy)




Epoch: 0, Loss: 1.388100266456604, Accuracy: 0.25820763087843834
Epoch: 1, Loss: 1.386849284172058, Accuracy: 0.25820763087843834


: 

In [ ]:

# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 25.91%
